# Playground for Exploring the Data

### Import packages

In [ ]:
from collections import OrderedDict
import json
import glob
import os

from settings import * 

### Start Playing Around

In [13]:
DATA_FOLDER_UNPROCESSED = f"{DATA_LOCAL_FOLDER}/signality/2023/Allsvenskan/"

# Find all frames parquet files
match_paths = glob.glob(os.path.join(DATA_FOLDER_UNPROCESSED, "*.json"))

# Initialize a set to store unique team names
unique_team_names = set()

# Iterate over each JSON file
for json_file in match_paths:
    # Load JSON data
    with open(json_file, 'r') as f:
        data = json.load(f, object_pairs_hook=OrderedDict)

    # Extract team_home_name
    team_home_name = data.get('team_home_name')
    
    # Add team_home_name to the set of unique team names
    if team_home_name:
        unique_team_names.add(team_home_name)

# Convert set to list for easier manipulation if needed
unique_team_names_list = sorted(list(unique_team_names))

# Print unique team names
# [print(team) for team in unique_team_names_list]

In [11]:
DATA_FOLDER_UNPROCESSED = f"{DATA_LOCAL_FOLDER}/signality/2023/Allsvenskan/"

# Find all JSON files
match_paths = glob.glob(os.path.join(DATA_FOLDER_UNPROCESSED, "*.json"))

# Iterate over each JSON file
for json_file in match_paths:
    # Load JSON data
    with open(json_file, 'r') as f:
        data = json.load(f, object_pairs_hook=OrderedDict)

    # Extract team_home_name and match ID
    team_home_name = data.get('team_home_name')
    match_id = data.get('id')
    
    # Check if Utsikten was the home team
    if team_home_name == "Utsikten":
        # Print match ID
        print("Match ID where Utsikten was the home team:", match_id)

Match ID where Utsikten was the home team: 98d877a9-0822-452a-bcac-93006226a84f


### Store as xlsx

In [ ]:
# Store frames_df as xslx
frames_df_head = frames_df.head(19979)

# Specify the file path for the Excel file
excel_file_path = f"{DATA_LOCAL_FOLDER}/Brommapojkarna_vs_Sirius.xlsx"

# Write the DataFrame to an Excel file
frames_df_head.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to {excel_file_path}")